# ETL

In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

## Carga `australian_user_reviewes.json` en el DataFrame `reviews`

In [2]:
# Esta funcion reemplaza "NaN" por "\"NaN\""
def reemplazar_nan_con_none(cadena):
    resultado = ""
    indice = 0
    longitud = len(cadena)

    while indice < longitud:
        if cadena[indice:indice+3] == "NaN":
            resultado += "\"NaN\""
            indice += 3
        else:
            resultado += cadena[indice]
            indice += 1

    return resultado

# Abre `australian_user_reviewes.json`, y crea el DataFrame `reviews`

with open('../dataset/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data_list = []
    for linea in file:
        linea = reemplazar_nan_con_none(linea)
        data = ast.literal_eval(linea.strip())
        if isinstance(data, dict):
            data_list.append(data)
    reviews = pd.DataFrame(data_list)


### Función analisis de sentimiento
Creación de columna `sentiment_analysis` aplicando análisis de sentimiento con NLP con la escala:

- `0` si es malo o reseña ausente
- `1` si es neutral
- `2` si es positivo

In [3]:
sia = SentimentIntensityAnalyzer()
def analyze_sentiment(lista):
    nlist =[]
    for i in lista:
        sentiment = sia.polarity_scores(i['review'])    
        try:               
            if sentiment['compound'] >= 0.05:
                i['sentiment_analysis'] = 2  # Positivo
                #del i['review']
            elif sentiment['compound'] <= -0.05:
                i['sentiment_analysis'] = 0  # Malo
                #del i['review']
            else:
                i['sentiment_analysis'] = 1  # Neutral
                #del i['review']
        except:
            i['sentiment_analysis'] = 1
            #del i['review']
        nlist.append(i)
    return nlist

In [4]:
reviews['reviews'] = reviews['reviews'].apply(lambda x: analyze_sentiment(x))   # Aplicando NLP

In [5]:
def desanidar_reviews(df):
    i = 0
    data_list = []
    while i <= len(df['user_id']) -1:
        user_id = df['user_id'].iloc[i]
        user_url = df['user_url'].iloc[i]
        lista = df['reviews'].iloc[i]
        for j in lista:
            j['user_id'] = user_id
            j['user_url'] = user_url
            data_list.append(j)
        i = i + 1
    return data_list

n_reviews = pd.DataFrame(desanidar_reviews(reviews))


Función para convertir fechas al formato "YYYY-MM-DD" para columna `posted`

In [6]:
def convertir_fecha(fecha):
    # Utilizar expresión regular para extraer componentes de la fecha
    match = re.match(r"Posted (\w+) (\d+), (\d+)", fecha)
    if match:
        mes_str, dia_str, anio_str = match.groups()
        # Mapear nombres de meses a números
        meses = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04',
            'May': '05', 'June': '06', 'July': '07', 'August': '08',
            'September': '09', 'October': '10', 'November': '11', 'December': '12'
        }
        # Crear la fecha en el nuevo formato
        nueva_fecha = f"{anio_str}-{meses[mes_str]}-{dia_str.zfill(2)}"
        return nueva_fecha
    else:
        return None
n_reviews['posted_date'] = n_reviews['posted'].apply(convertir_fecha)
n_reviews['posted_date'] = pd.to_datetime(n_reviews['posted_date'])
n_reviews = n_reviews.drop(['posted'], axis=1)

Función que permite normalizar la fecha para la columna `last_edited`

In [7]:
def convertir_fecha(fecha):
    # Utilizar expresión regular para extraer componentes de la fecha
    match = re.match(r"Last edited (\w+) (\d+), (\d+)", fecha)
    if match:
        mes_str, dia_str, anio_str = match.groups()
        # Mapear nombres de meses a números
        meses = {
            'January': '01', 'February': '02', 'March': '03', 'April': '04',
            'May': '05', 'June': '06', 'July': '07', 'August': '08',
            'September': '09', 'October': '10', 'November': '11', 'December': '12'
        }
        # Crear la fecha en el nuevo formato
        nueva_fecha = f"{anio_str}-{meses[mes_str]}-{dia_str.zfill(2)}"
        return nueva_fecha
    else:
        return None

n_reviews['last_edited'] = n_reviews['last_edited'].apply(convertir_fecha)
n_reviews['last_edited'] = pd.to_datetime(n_reviews['last_edited'])

Función que permite extraer a la cantidad de personas que cosidenraron graciosa las review de la columna `funny`

In [8]:
def extraer_numero(cadena):
    match = re.match(r"([\d,]+)", cadena)
    if match:
        numero_str = match.group(1).replace(',', '')
        return int(numero_str)
    else:
        return 0
    
n_reviews['funny'] = n_reviews['funny'].apply(extraer_numero)
n_reviews['funny'] = n_reviews['funny'].astype(int)

Eliminar columna `user_url`

In [9]:
n_reviews = n_reviews.drop(['user_url'], axis=1)

Extraer votos realizados a las reviews realizadas de la columna `helpful`, se extraen  de la siguiente manera:
- `Useful_recommend` Cantidad de votos positivos
- `#_recommend` Total de votos
- `%_recommend` Porcentaje de votos positivos

In [10]:
def extraer_numeros(cadena):
    match = re.match(r"(\d+) of (\d+) people \((\d+)%\) found this review helpful", cadena)
    if match:
        serie = pd.Series({
            'Useful_recommend': int(match.group(1)),  # 15
            '#_recommend': int(match.group(2)),  # 20
            '%_recommend': int(match.group(3))   # 75
        })
        return serie
    else:
        return pd.Series({'Useful_recommend': 0, '#_recommend': 0, '%_recommend': 0})


n_reviews[['Useful_recommend','#_recommend','%_recommend']] = n_reviews['helpful'].apply(extraer_numeros)
n_reviews['%_recommend'] = n_reviews['%_recommend'].astype(float) / 100
n_reviews[['Useful_recommend','#_recommend']] = n_reviews[['Useful_recommend','#_recommend']].astype(int)
n_reviews = n_reviews.drop(['helpful'], axis=1)

Las reviews son almacenadas en el archivo `reviews.parquet`

In [11]:
n_reviews['item_id'] = n_reviews['item_id'].astype(int)
n_reviews.to_parquet('../dataset/reviews.parquet', engine='pyarrow', compression='snappy')

Creación de DataFrame `df_vectores_reviews` para la función def `recomendacion_juego`. Este DataFrame contiene las recomendaciónes por `item_id` de cada videojuego y se vectoriza por feature words, el resultado se almacena en el archivo `reviews_per_item.parquet`

In [12]:
data = {}
for i in n_reviews.iterrows():
    if n_reviews['item_id'].iloc[i[0]] in data:
        lista = data[n_reviews['item_id'].iloc[i[0]]]
        lista.append(n_reviews['review'].iloc[i[0]])
    else:
        lista = []
        lista.append(n_reviews['review'].iloc[i[0]])
        data[n_reviews['item_id'].iloc[i[0]]]=lista
reviews = pd.DataFrame(list(data.items()), columns=['id', 'review'])

reviews['reviews_concatenadas'] = reviews['review'].apply(lambda x: ' '.join(x))

vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)  # Se eligen 1000 palabras claves para vectorizar
# Aplicar el vectorizador a los textos de las reviews
vectores_reviews = vectorizer.fit_transform(reviews['reviews_concatenadas'])

df_vectores_reviews = pd.concat([reviews[['id', 'reviews_concatenadas']], pd.DataFrame(vectores_reviews.toarray(), columns=[f"feature_{i}" for i in range(vectores_reviews.shape[1])])], axis=1)
df_vectores_reviews = df_vectores_reviews.drop(['reviews_concatenadas'], axis=1)
df_vectores_reviews['id'] = df_vectores_reviews['id'].astype(int)
# Guardar el DataFrame en formato Parquet
df_vectores_reviews.to_parquet('../dataset/reviews_per_item.parquet', engine='pyarrow', compression='snappy')

Creación de DataFrame que alimentará la función `recomendacion_juego` almacenada en `game_recommend.parquet`.

In [13]:
games_recommend = n_reviews[['item_id','sentiment_analysis']]
games_recommend = games_recommend.dropna()
games_recommend = games_recommend.fillna(0)
games_recommend.to_parquet('../dataset/games_recommend.parquet', engine='pyarrow', compression='snappy')

## Carga `output_steam_games.json` en el DataFrame `games`

In [14]:
with open('../dataset/output_steam_games.json', 'r', encoding='utf-8') as file:
    data_list = []
    for linea in file:
        linea = reemplazar_nan_con_none(linea)
        data = json.loads(linea.strip())
        if isinstance(data, dict):
            data_list.append(data)
    games = pd.DataFrame(data_list)

Eliminando registros nulos

In [15]:
games = games[games['id'] != 'NaN']

Normlizando fecha para la columna `release_date`

In [16]:
games.loc[games['release_date'] == 'Soon..', 'release_date'] = pd.NaT 
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')

Inputando valor de `0` para cada registro que contenga `str` de la columna `price`

In [17]:
i = 0
while i<len(games['price']):
    if type(games['price'].iloc[i]) == str:
        games['price'] = games['price'].replace(games['price'].iloc[i],0)
    i += 1
games['price'] = games['price'].replace('NaN', np.nan)

La columna `early_access` se formatea como tipo booleano

In [18]:
games['early_access'] = games['early_access'].astype(bool)

La columna `id` es formateada como dato tipo entero

In [19]:
games['id'] = games['id'].astype(int)

El DataFrame es almacenado en el archivo `games.parquet`

In [20]:
games = games.replace('NaN', np.nan)
games.to_parquet('../dataset/games.parquet', engine='pyarrow', compression='snappy')

Creacion de DataFrame para alimentar la función `UsersRecommend`, el cual es almacenado en `recommend.parquet`

In [21]:
n_reviews['year'] = n_reviews['posted_date'].dt.year
recommend = n_reviews.merge(games, left_on='item_id', right_on='id', how='inner')
filtro = recommend[recommend['recommend'] == True]
recommend = filtro.groupby(['title','year'])['recommend'].value_counts().reset_index()
recommend.to_parquet('../dataset/recommend.parquet', engine='pyarrow', compression='snappy')

Creacion de DataFrame para la función `UsersNotRecommend`, que es almacenado en `notrecommend.parquet`

In [22]:
notrecommend = n_reviews.merge(games, left_on='item_id', right_on='id', how='inner')
notrecommend['not_recommend'] = (notrecommend['recommend'] == False) | (notrecommend['sentiment_analysis'] == 0)
filtro1 = notrecommend[notrecommend['not_recommend'] == True]
notrecommend = filtro1.groupby(['title','year'])['not_recommend'].value_counts().reset_index()
notrecommend.to_parquet('../dataset/notrecommend.parquet', engine='pyarrow', compression='snappy')

Creación de DataFrame para la función `sentiment_analysis` que es almacenado en el archivo `sentiments.parquet`

In [23]:
merged_sentiments = games.merge(n_reviews, left_on='id', right_on='item_id', how='inner')
sentiments = merged_sentiments[['release_date','sentiment_analysis']]
sentiments['release_date'] = pd.to_datetime(sentiments['release_date'], format="%Y-%m-%d")
sentiments.to_parquet('../dataset/sentiments.parquet', engine='pyarrow', compression='snappy')

C:\Users\stdio\AppData\Local\Temp\ipykernel_2484\505952917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiments['release_date'] = pd.to_datetime(sentiments['release_date'], format="%Y-%m-%d")


## Carga `australian_users_items.json` en el DataFrame `items`

In [24]:
def extraer_valores(cadena):
    valores = {}
    
    # Expresiones regulares para cada clave
    patrones = {
        'user_id': r'"user_id"\s*:\s*"([^"]+)"',
        'items_count': r'"items_count"\s*:\s*([^,]+)',
        'steam_id': r'"steam_id"\s*:\s*"([^"]+)"',
        'user_url': r'"user_url"\s*:\s*"([^"]+)"',
        'items': r'"items"\s*:\s*\{([^}]+)\}'
    }
    
    for clave, patron in patrones.items():
        coincidencias = re.search(patron, cadena)
        if coincidencias:
            valores[clave] = coincidencias.group(1)
    
    return valores


def procesar_diccionarios(items_parte):
    # Utiliza una expresión regular para encontrar todas las coincidencias de los diccionarios dentro de corchetes.
    diccionarios = re.findall(r'\{[^}]+\}', items_parte)

    # Inicializa una lista para almacenar los diccionarios procesados.
    resultado = []

    # Define una función para procesar cada diccionario.
    def procesar_diccionario(diccionario_str):
        # Utiliza una expresión regular para extraer los valores de las claves deseadas.
        item_id_match = re.search(r'"item_id": "([^"]+)"', diccionario_str)
        item_name_match = re.search(r'"item_name": "([^"]+)"', diccionario_str)
        playtime_forever_match = re.search(r'"playtime_forever": (\d+)', diccionario_str)
        playtime_2weeks_match = re.search(r'"playtime_2weeks": (\d+)', diccionario_str)

        # Verifica si se encontró una coincidencia para cada clave antes de extraer el valor.
        item_id = item_id_match.group(1) if item_id_match else None
        item_name = item_name_match.group(1) if item_name_match else None
        playtime_forever = int(playtime_forever_match.group(1)) if playtime_forever_match else None
        playtime_2weeks = int(playtime_2weeks_match.group(1)) if playtime_2weeks_match else None

        # Crea un diccionario con los valores extraídos.
        diccionario_resultado = {
            "item_id": item_id,
            "item_name": item_name,
            "playtime_forever": playtime_forever,
            "playtime_2weeks": playtime_2weeks
        }

        return diccionario_resultado

    # Procesa cada diccionario encontrado y agrégalo a la lista de resultados.
    for diccionario_str in diccionarios:
        resultado.append(procesar_diccionario(diccionario_str))

    # El resultado es una lista de diccionarios.
    return resultado




items = pd.DataFrame(columns=['user_id', 'items_count','steam_id','user_url','items'])
with open('../dataset/australian_users_items.json', 'r', encoding='utf-8') as file:
    data_list = []
    for linea in file:
        linea = reemplazar_nan_con_none(linea)
        linea = linea.replace('\'', '\"')
        resultado = re.search(r'"items":\s*(.+)', linea)
        linea = extraer_valores(linea)
        items_parte = resultado.group(1)
        items_parte = procesar_diccionarios(items_parte)
        linea['items'] = items_parte 
        data_list.append(linea)

items = pd.DataFrame(data_list)

In [25]:
# Revisando registros duplicados
items['user_id'].value_counts()

user_id
X03-Suits            3
76561198027488037    3
76561198100326818    3
76561198309337430    3
76561198051777058    3
                    ..
8392158              1
76561198056804863    1
SparklezTheTurtle    1
76561198019707497    1
edward_tremethick    1
Name: count, Length: 87626, dtype: int64

In [26]:
# Eliminar registros duplicados
items = items.drop_duplicates(subset=['user_id']) 

Función cuyo proposito es desanidar el contenido de la columna `items`

In [27]:
def desanidar_items(df):
    i = 0
    data_list = []
    while i <= len(df['user_id']) - 1:
        user_id = df['user_id'].iloc[i]
        steam_id = df['steam_id'].iloc[i]
        user_url = df['user_url'].iloc[i]
        lista = df['items'].iloc[i]
        for j in lista:
            j['user_id'] = user_id
            j['steam_id'] = steam_id
            j['user_url'] = user_url
            data_list.append(j)
        i = i + 1
    return data_list

Creación de un nuevo Dataframe `n_items`

In [28]:
n_items = pd.DataFrame(desanidar_items(items))

In [29]:
n_items[['item_id']] = n_items[['item_id']].astype(int)
n_items = n_items.drop(['user_url'], axis=1)

In [30]:
n_items.to_parquet('../dataset/items.parquet', engine='pyarrow', compression='snappy')

Creacion de DataFrame para alimentar la función `PlayTimeGenre` la cual se almacenara en el archivo `ranking_genre.parquet` con el objeto de economizar en espacio de memoria RAM

In [31]:
# No ejecutar esta celda en render ya que se consume alrededor de 2 Gb de memoria y la versión gratis tiene solo 512 Mb
# Separar lista de generos
games_exploded = games.explode('genres')
# Reemplaza las cadenas que no sean fechas válidas por NaN
games_exploded['release_date'] = pd.to_datetime(games_exploded['release_date'], errors='coerce')
#Crear columna year
games_exploded['year'] = games_exploded['release_date'].dt.year
# Combinar DF's games_explode con items coincidiendo por id    
merged_data = games_exploded.merge(n_items, left_on='id', right_on='item_id', how='inner')
# Sumar playtime_forever por género y año
genre_playtime = merged_data.groupby(['genres','year'])['playtime_forever'].sum().reset_index()
# Ordenar de mayor a menor sobre 'playtime_forever'
ranking_genre = genre_playtime.sort_values(by='playtime_forever', ascending=False)
ranking_genre = ranking_genre.reset_index(drop=True)
# Guardar df en formato parquet
ranking_genre.to_parquet('../dataset/ranking_genre.parquet', engine='pyarrow', compression='snappy')

Creación de DataFrame para alimentar la funcion `UserForGenre` la cual se almacena en el archivo `user_genre.parquet`

In [32]:
# Crear df user_genre.parquet

# Creando df con las columnas 'genres', 'playtime_forever' y 'user_id_y'
usersXgenre = merged_data[['genres','playtime_forever','user_id','year']]
# Agrupando por usuario y genero
usersXgenre = usersXgenre.groupby(['genres','user_id','year'])['playtime_forever'].sum().reset_index()
# Guardar df en formato parquet
usersXgenre.to_parquet('../dataset/user_genre.parquet', engine='pyarrow', compression='snappy')